In [1]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import time

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.001
epochs = 50
batch_size = 32

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 128 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [3]:
import tensorflow as tf

In [4]:
#Input Placeholders 
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

In [5]:
# Create model
def multilayer_perceptron(x, n_input, n_hidden_1, n_hidden_2, n_classes):
    # Hidden layer with RELU activation
    with tf.name_scope("layer1"):
        w1= tf.Variable(tf.random_normal([n_input, n_hidden_1]), name = "weights")
        b1 = tf.Variable(tf.random_normal([n_hidden_1]), name = "bias")
        layer_1 = tf.add(tf.matmul(x,w1), b1)
        layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    with tf.name_scope("hidden_layer"):
        w2 =  tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name = "weights")
        b2 =  tf.Variable(tf.random_normal([n_hidden_2]), name = "bias")
        layer_2 = tf.add(tf.matmul(layer_1, w2), b2)
        layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    with tf.name_scope("output_layer"):
        w_o = tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name = "weights")
        b_o = tf.Variable(tf.random_normal([n_classes]), name = "bias")
        out_layer = tf.matmul(layer_2, w_o) + b_o
    return out_layer

In [6]:
# Construct model
pred = multilayer_perceptron(x, n_input, n_hidden_1, n_hidden_2, n_classes)

# Minimize error using cross entropy
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, labels=y))
# Gradient Descent
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


with tf.name_scope("prediction"):
    prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y,1))
with tf.name_scope("accuracy"):
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
tf.summary.scalar("accuracy", accuracy)

# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)
merged_summary_op = tf.summary.merge_all()

In [7]:
# We can see all the operations embedded to the graph. You can see how scope helps in naming an operation at the end.
sess = tf.Session()
op = sess.graph.get_operations()
[m.values() for m in op]

[(<tf.Tensor 'Placeholder:0' shape=(?, 784) dtype=float32>,),
 (<tf.Tensor 'Placeholder_1:0' shape=(?, 10) dtype=float32>,),
 (<tf.Tensor 'layer1/random_normal/shape:0' shape=(2,) dtype=int32>,),
 (<tf.Tensor 'layer1/random_normal/mean:0' shape=() dtype=float32>,),
 (<tf.Tensor 'layer1/random_normal/stddev:0' shape=() dtype=float32>,),
 (<tf.Tensor 'layer1/random_normal/RandomStandardNormal:0' shape=(784, 256) dtype=float32>,),
 (<tf.Tensor 'layer1/random_normal/mul:0' shape=(784, 256) dtype=float32>,),
 (<tf.Tensor 'layer1/random_normal:0' shape=(784, 256) dtype=float32>,),
 (<tf.Tensor 'layer1/weights:0' shape=(784, 256) dtype=float32_ref>,),
 (<tf.Tensor 'layer1/weights/Assign:0' shape=(784, 256) dtype=float32_ref>,),
 (<tf.Tensor 'layer1/weights/read:0' shape=(784, 256) dtype=float32>,),
 (<tf.Tensor 'layer1/random_normal_1/shape:0' shape=(1,) dtype=int32>,),
 (<tf.Tensor 'layer1/random_normal_1/mean:0' shape=() dtype=float32>,),
 (<tf.Tensor 'layer1/random_normal_1/stddev:0' shape

In [8]:
logs_path = '/tmp/tensorflowlogs/FNN'

In [9]:
# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)
merged_summary_op = tf.summary.merge_all()

In [10]:
start = time.time()
with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.summary.FileWriter(logs_path, graph = sess.graph)
    step = 0
    for i in range(epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        for j in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            m, _, summary = sess.run([cost, optimizer, merged_summary_op], feed_dict={x: batch_xs,y: batch_ys})
            summary_writer.add_summary(summary, step)
            step += 1 
            if j % 500 == 0:
                train_acc = sess.run([accuracy], feed_dict={x: mnist.train.images, y: mnist.train.labels})
                valid_acc = sess.run([accuracy], feed_dict={x: mnist.validation.images, y: mnist.validation.labels})
                print ("m: ", m , "Train_Accuracy: ", train_acc, "Validation_Accuracy: ", valid_acc, "time_taken:", (time.time() - start))
    print ("total time taken: ", time.time()-start)
    test_accuracy = sess.run([accuracy],feed_dict={x: mnist.test.images, y: mnist.test.labels} )
    print("Test Accuracy:", test_accuracy)

m:  478.418 Train_Accuracy:  [0.12798181] Validation_Accuracy:  [0.1348] time_taken: 2.9023070335388184
m:  16.8129 Train_Accuracy:  [0.74827272] Validation_Accuracy:  [0.74779999] time_taken: 6.3382790088653564
m:  4.22055 Train_Accuracy:  [0.84016365] Validation_Accuracy:  [0.83920002] time_taken: 9.73911714553833
m:  8.39593 Train_Accuracy:  [0.8737818] Validation_Accuracy:  [0.87080002] time_taken: 12.911126136779785
m:  2.97097 Train_Accuracy:  [0.88114548] Validation_Accuracy:  [0.87840003] time_taken: 14.97954511642456
m:  5.25204 Train_Accuracy:  [0.89896363] Validation_Accuracy:  [0.89480001] time_taken: 18.15337610244751
m:  3.76669 Train_Accuracy:  [0.91036361] Validation_Accuracy:  [0.90420002] time_taken: 21.2986102104187
m:  0.469164 Train_Accuracy:  [0.92083639] Validation_Accuracy:  [0.91540003] time_taken: 24.486225128173828
m:  5.3037 Train_Accuracy:  [0.92252725] Validation_Accuracy:  [0.917] time_taken: 26.569631099700928
m:  0.302971 Train_Accuracy:  [0.93010908] V

In [11]:
! tensorboard --logdir=/tmp/tensorflowlogs/FNN

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.5/bin/tensorboard", line 7, in <module>
    from tensorflow.tensorboard.tensorboard import main
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/__init__.py", line 23, in <module>
    from tensorflow.python import *
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/python/__init__.py", line 48, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow', fp, pathname, description)
  File "/Library/Framework